In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('appName').setMaster('local[*]')
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=appName>

In [2]:
### PYTHON CLASS SERIALIZING JSON
import json
from json import JSONEncoder

class Pessoa():
    def __init__(self, nome, sobre_nome, num_cpf, num_identidade, dth_nascimento, endereco_pessoa, veiculo):
        self.nome = nome
        self.sobre_nome = sobre_nome
        self.num_cpf = num_cpf
        self.num_identidade = num_identidade
        self.dth_nascimento = dth_nascimento
        self.endereco_pessoa = endereco_pessoa
        self.veiculo = veiculo
        
class Endereco():
    def __init__(self, num_cep, cidade, uf, endereco, num_endereco, complemento):
        self.num_cep = num_cep
        self.cidade = cidade
        self.uf = uf
        self.endereco = endereco
        self.num_endereco = num_endereco
        self.complemento = complemento
    
class Veiculo():
    def __init__(self, marca, modelo, tipo, ano):
        self.tipo = tipo
        self.marca = marca
        self.modelo = modelo
        self.ano = ano

class Produto():
    def __init__(self, nome_produto, cod_produto, desc_produto):
        self.nome_produto = nome_produto
        self.cod_produto = cod_produto
        self.desc_produto = desc_produto
        
class Contrato():
    def __init__(self, duracao, valor, parcelado, valor_parcela, desconto, valor_desconto,
                 valor_total,ini_vigencia,fin_vigencia, dth_compra,num_apolice, produto, pessoa):
        self.duracao = duracao
        self.valor = valor
        self.parcelado = parcelado
        self.valor_parcela = valor_parcela
        self.desconto = desconto
        self.valor_desconto = valor_desconto
        self.valor_total = valor_total
        self.ini_vigencia = ini_vigencia
        self.fin_vigencia = fin_vigencia
        self.dth_compra = dth_compra
        self.num_apolice = num_apolice
        self.produto = produto
        self.pessoa = pessoa
        
#subclass JSONEncoder
class SeguroEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [3]:
import random
import time
import hashlib
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO

def str_time_prop(start, end, format, prop):
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(format, time.localtime(ptime))

### Format date para o elasticsearch entender o field gerado como date
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y/%m/%d %I:%M:%S', prop)

### Gera número de apólice randômica
def random_num_apolice():
    return random.randint(200000000000000,399999999999999)

### Gera hash através da data e código para criar a chave _id para o documento no indice
def random_hash(timestamp, cod):
    cod_h = timestamp + cod
    return str(hashlib.md5(cod_h.encode()).hexdigest())

### Gera numero de CPF randomicamente usando um range padrão
def random_cpf():
    return random.randint(20000000001,88782514100)

### Gerador veiculo
def random_veiculo():
    marcas = ['VW','TOYOTA','GM', 'FORD', 'HONDA']
    modelos = [['jetta', 't-cross', 'amarok'],['camry', 'rav4', 'hilux'],
               ['prisma', 'cruze', 's10'],['fusion','ecosport','ranger'],['civic', 'hrv','ridgeline']]
    tipos = ['passeio','SUV','pick-up']
    
    marca_selected = random.choice(marcas)
    ind_marca = marcas.index(marca_selected)
    modelo = random.choice(modelos[ind_marca])
    ind_modelo = modelos[ind_marca].index(modelo)
    tipo = tipos[ind_modelo]
    veiculo = [marca_selected, modelo, tipo, random.randint(2018,2020)]
    return veiculo

### Gerador endereco
def random_endereco():
    uf_array = ['DF', 'SP', 'RJ', 'GO', 'RS', 'PR', 'SC', 'BA', 'TO' ]
    uf_rand = random.choice(uf_array)
    cep_rand = random.randint(10000000,99999999)
    endreco = [uf_rand, cep_rand]
    return endreco

### Gerador Produto
def random_produto():
    produtos = [['1021','Seguro auto premium','Seguro super premium de auto'], ['10111','Seguro auto light','Seguro auto simples'],
                        ['2020','Seguro auto executive', 'Seguro para executivos'], ['10222','Seguro auto total', 'Seguro total nacional']]
    produto_arr = random.choice(produtos)
    return produto_arr
### mudar a data de inicio da vigencia para ter um resultado próximo aos dias atuais.
def random_contrato():
    duracao = 12
    valor = random.randint(1200,3000)
    parcelado = "true"
    valor_parcela = round((valor / duracao), 2)
    desconto = "false"
    valor_desconto = 0.0
    valor_total = round((valor_parcela * duracao), 2)
    ini_vigencia = random_date("2019/12/1 1:30:00", "2020/7/31 1:30:00", random.random())
    date_time_obj = datetime.strptime(ini_vigencia, '%Y/%m/%d %H:%M:%S') + relativedelta(months =+12)
    fin_vigencia = str(date_time_obj)
    dth_compra = ini_vigencia
    num_apolice = random_num_apolice()
    return [duracao, valor, parcelado, valor_parcela, desconto, valor_desconto,
                 valor_total,ini_vigencia,fin_vigencia, dth_compra,int(num_apolice)]
    

In [4]:
from collections import defaultdict
from datetime import datetime
import random
import json

def gerador_contratos(): 
    ### Geradores randomicos
    veiculo_rand = random_veiculo()
    dt_nascimento = random_date("1975/1/1 1:30:00", "2000/12/31 1:30:00", random.random())
    endereco_rand = random_endereco()
    produto_rand = random_produto()
    contrato_rand = random_contrato()
    ### Pupular classes para renderizar o json
    veiculo = Veiculo(veiculo_rand[0],veiculo_rand[1],veiculo_rand[2], veiculo_rand[3])
    endereco_pessoa = Endereco(endereco_rand[1], 'xxxxx', endereco_rand[0], 'Rua x apsd', 1, 'ap 82' )
    pessoa = Pessoa('MMM', 'SILVA', random_cpf(), 123213, dt_nascimento , endereco_pessoa, veiculo)
    produto = Produto(produto_rand[1], produto_rand[0], produto_rand[2])
    contrato = Contrato(contrato_rand[0], contrato_rand[1], contrato_rand[2], contrato_rand[3], contrato_rand[4], contrato_rand[5],
                     contrato_rand[6],contrato_rand[7],contrato_rand[8], contrato_rand[9],contrato_rand[10], produto, pessoa)
    ### Renderizar o json de contrato
    ## contratoJSON = json.dumps(contrato, indent=4, cls=SeguroEncoder)
    contratoJSON = json.dumps(contrato, cls=SeguroEncoder)
    return contratoJSON

### Função de formatar a lista para o gerar o rdd
def format_data(x):
    return(x['num_apolice'], json.dumps(x))


In [5]:
### Gerador de contratos
## Quantidade de contratos desejados
qtd_contrato = 10000
event_list = []
while qtd_contrato > 0:
    event_list.append(json.loads(gerador_contratos()))
    qtd_contrato = qtd_contrato - 1


In [10]:
## Gera do RDD para armazenamento no elasticsearch
## A 
rdd = sc.parallelize(map(lambda x: format_data(x),event_list))

## Configurações para persistência no elasticsearch
conf = {
    "es.nodes" : 'https://172.17.0.2',
    "es.port" : '9200',
    "es.net.ssl" : 'true',
    "es.resource" : 'eventos',
    "es.input.json" : 'yes',
    "es.mapping.id": 'num_apolice',
    "es.net.http.auth.user" :'admin',
    "es.net.http.auth.pass" : 'admin'
}

In [11]:
## executa o método de persistencia no elasticsearch de acordo com as configurações para persistência.
rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=conf)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsNewAPIHadoopFile.
: org.elasticsearch.hadoop.EsHadoopIllegalArgumentException: Cannot detect ES version - typically this happens if the network/Elasticsearch cluster is not accessible or when targeting a WAN/Cloud instance without the proper setting 'es.nodes.wan.only'
	at org.elasticsearch.hadoop.rest.InitializationUtils.discoverClusterInfo(InitializationUtils.java:340)
	at org.elasticsearch.hadoop.mr.EsOutputFormat.init(EsOutputFormat.java:262)
	at org.elasticsearch.hadoop.mr.EsOutputFormat.checkOutputSpecs(EsOutputFormat.java:235)
	at org.apache.spark.internal.io.HadoopMapReduceWriteConfigUtil.assertConf(SparkHadoopWriter.scala:393)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsNewAPIHadoopDataset$1(PairRDDFunctions.scala:1077)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.jav